In [12]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [13]:
sigmoid = lambda z : 1 /(1+np.exp(-z))
dsigmoid = lambda x : sigmoid(x) * (1-sigmoid(x))

In [14]:
class NN:
    def __init__(self, ni, nh, no, bias=1.0):
        self.ni = ni
        self.nh = nh
        self.no = no
        self.b = bias

        self.ah = [1.0] * nh
        self.ao = [1.0] * no

        self.wh = np.random.rand(self.nh, self.ni)
        self.wo = np.random.rand(self.no, self.nh)

        self.ch = np.zeros((self.nh, self.ni))
        self.co = np.zeros((self.no, self.nh))

    def ff(self, inp, b=1.0):
        #hidden activations
        t =  1.0 + self.wh.dot(inp)
        g = sigmoid(t)
        self.ah = g

        #output activations
        t = 1.0 + self.wo.dot(g)
        out = np.tanh(t)
        self.ao = out
        return out

    def bp(self, x, y, N=0.5, M=0.1):
        #N = learning rate
		#M = Momentum
        nn_out = self.ff(x)

        #output layer errors
        outerror = y - self.ao
        dzj = dsigmoid(self.ao) * outerror
        """for i in range(self.no):
            error = y[i]-self.ao[i]
            dzj[i] = dsigmoid(self.ao[i]) * error"""

        #hidden layer errors
        herror = dzj.dot(self.wo)
        dyi = dsigmoid(self.ah) * herror
        """for i in range(self.nh):
            error = 0.0
            for j in range(self.no):
                error += dzj[j] * self.wh[i][j]
            dyi[i] = dsigmoid(self.ah[i]) * error"""

        #update weights output first, then hidden
        for i in range(self.no):
            for j in range(self.nh):
                change = dzj[i] * self.ah[j]
                self.wo[i][j] += N * change + M * self.co[i][j]
                self.co[i][j] = change

        for i in range(self.nh):
            for j in range(self.ni):
                change = dyi[i] * x[j]
                self.wh[i][j] += N * change + M * self.ch[i][j]
                self.ch[i][j] = change

    def train(self, x, y, epochs=24, batch=100):
        xtrain = x[:int(len(x)*.7)]
        ytrain = y[:int(len(y)*.7)]
        xtest = x[int(len(x)*.7)::]
        ytest = y[int(len(x)*.7)::]
        for i in range(epochs):
            print ("\r" + str(i+1) + "/" + str(epochs) + " Epochs", end="")
            self.bp(xtrain[i], ytrain[i])
            #get batch size and bp
            """mini_batches = [xtrain[i:i+batch] for k in range(0, len(xtrain), batch)]
            mini_y = [ytrain[i] for i in range(0, len(ytrain), batch)]
            for mini_batch in mini_batches:
                self.batch_calculate(mini_batch, mini_y)"""
        out = self.ff(xtest[1])
        print (out)
        print (str(ytest[1]))
        """for i in range(len(xtest)):
			out = self.ff(xtest[i])
			print (out) 
			print (str(ytest[i]))
			print (" ")"""

In [15]:
nn = NN(784, 24, 10)
print ("Loading Data")
dat = pd.read_csv('train.csv', sep=',', header=0).values
trainx = np.array([np.array(i[1::]) for i in dat])
trainy = np.array([float(i[0]) for i in dat])

ty = []
for i in range(len(trainy)):
	test = [0.0] * 10
	test[int(trainy[i])] = 1.0
	ty.append(np.array(test))
trainy = np.array(ty)

nn.train(trainx, trainy, epochs=5)
print ("done")

Loading Data
5/5 Epochs[ 0.76159416  0.76159416  0.76159416  0.76159416  0.76159416  0.76159416
  0.76159416  0.76159416  0.76159416  0.76159416]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
done
